In [1]:
import numpy as np
import math
from PIL import Image
Image.MAX_IMAGE_PIXELS = None # working with very high def images

In [2]:
# just for my use
import timeit

In [3]:
mercury0 = 'pics/mercury1.png'

In [4]:
moon0 = 'pics/moonbump4k.jpg'

In [5]:
def load_data(img_path = mercury0):
    ''' 
    A function that loads an image from a file and converts it into a 1D array.
    
    Parameters
    ----------
    img_path: String, optional
        String file path to image location (default demo file)   
        
    Returns
    -------
    img1D: array 
        1D array of original pixel values
    '''
    
    image = Image.open(img_path)
    img1D = np.array(image.getdata())
     
    return img1D

In [58]:
def scale_data(img_path = mercury0, min_thresh = 0, max_thresh = None, protrusion = 10.0, max_is_max = True):
    ''' 
    A function that loads the 1D pixel data and scales it appropriately.
    
    ### AUDREY: should maybe just load 2D array and then flatten it?         
    ## also take true false for if greater height is white or black (max or min)           
    '''
    
    image = Image.open(img_path)
    img1D = np.array(image.getdata())
    
    if max_thresh == None:
        max_height = np.max(img1D)
    else:
        max_height = max_thresh
        img1D[img1D > max_height] = max_height # set all vals abolve max to max
        
    if min_thresh != 0: 
        min_height = min_thresh
    else:
        min_height = np.min(img1D)
        img1D = img1D - min_height # overwrite, subtract floor from all vals
        
    if (max_height == min_height): # try to avoid dividing by zero
        height_scalar = 1
    else:
        height_scalar = protrusion / abs(max_height - min_height)
    
    img1D = img1D * height_scalar # now on a scale from 0 to 10 mm
    
    img2D = np.reshape(img1D, (image.size[1], image.size[0])) # finally shape the array into an image

    return img2D


In [37]:
img = scale_data()

In [28]:
mercury0_p5 = scale_data(protrusion = 5)

In [7]:
img_mercury_thresh100 = scale_data(max_thresh = 100)

In [76]:
img_moon0 = scale_data(img_path = moon0)

In [30]:
apex, base, big_cols = define_shape()

In [89]:
apex, base, big_cols = define_shape(resolution = 100)

In [31]:
# calculator, aligns the heights with the tiangular sphere coords

def data_overlay(image, icos_apex = apex, icos_base = base): # AUDREY PLUG IN SHAPE PLOT DATA HERE I THINK
    
    x_pixels = len(image[0])
    y_pixels = len(image)

    x_scale = x_pixels / 360 # for theta
    y_scale = y_pixels / 180 # for phi
    
    print("x:", x_pixels, "y:", y_pixels)
    
    sphere_radius = apex[0] # all radii should be the same right now
    
    # go vertex by vertex of triangle, ask the dataset what height it should be at, and push vertex to that height    
    
    grid_deg = big_cols # AUDREY FITURE OUT WHO NEEDS WHAT IN WHAT ORDER PLS
    grid_px = np.zeros((len(grid_deg), len(grid_deg[0]), len(grid_deg[0][0])), dtype = np.ndarray) # pixel height array
    # img = scale_data() # AUDREY DECIDE IF YOU'LL DO THIS
    
    for ii in range (len(grid_deg)): # which of the 5 big columns
        for jj in range (len((grid_deg[0]))): # which row of a big column
            for kk in range (len((grid_deg[0][0]))): # which entry (real column)
                
                # find the pixels this sphere thing corresponds to 
                
                theta = grid_deg[ii][jj][kk][1] 
                phi = grid_deg[ii][jj][kk][2]
                
                x = int(math.floor(theta * x_scale) - 1) # turn it into an index to get a pixel from
                y = int(math.floor(phi * y_scale) - 1)
                
                radius = image[y][x] + sphere_radius # we calculated this earlier, add to base radius of sphere
                
                sphere_coord = [radius, theta, phi]
                cart_coord = spherical_to_cartesian(sphere_coord)
                
                # fill in pixel grid
                grid_px[ii][jj][kk] = cart_coord
                
                # AUDREY FIX THE APEX AND BASE AND HAND THEM BACK!
    
    
    # now do the apex and the base    
    apex_x = int(math.floor(icos_apex[1] * x_scale) - 1) # turn it into an index to get a pixel from
    apex_y = int(math.floor(icos_apex[2] * y_scale) - 1)
                
    apex_radius = image[apex_y][apex_x] + sphere_radius # we calculated this earlier
                
    apex_sphere = [apex_radius, icos_apex[1], icos_apex[2]]
    apex_cart = spherical_to_cartesian(apex_sphere)
    
    base_x = int(math.floor(icos_base[1] * x_scale) - 1) # turn it into an index to get a pixel from
    base_y = int(math.floor(icos_base[2] * y_scale) - 1)
                
    base_radius = image[base_y][base_x] + sphere_radius # we calculated this earlier
                
    base_sphere = [base_radius, icos_base[1], icos_base[2]]
    base_cart = spherical_to_cartesian(base_sphere)
    
    return grid_px, apex_cart, base_cart                                

In [79]:
mesh_moon0, apex_moon0, base_moon0 = data_overlay(image = img_moon0)

x: 4000 y: 2000


In [82]:
mesh_merc100, apex_merc100, base_merc100 = data_overlay(image = img_mercury_thresh100)

x: 23040 y: 11520


In [32]:
# RUN THIS LINE SECOND

mesh, apex, base = data_overlay(image = mercury0_p5)

x: 23040 y: 11520


# Supporting Functions

In [10]:
# spherical to cartesian coordinate converter

def spherical_to_cartesian(sphere_coord): 
    
    radius = sphere_coord[0]
    theta = sphere_coord[1] * math.pi / 180
    phi = sphere_coord[2] * math.pi / 180
    
    x = radius * math.sin(phi) * math.cos(theta)
    y = radius * math.sin(phi) * math.sin(theta)
    z = radius * math.cos(phi)
    
    cart_coord = [x, y, z]
    
    return cart_coord

In [18]:
def stitch_triangles(A, B, C, D, lines): # A,B,C,D are the defined points of a parallelogram
    # lines is the list to write strings of the file to
    
    #   A . B
    #    .   .   < = my parallelogram :)
    #     C . D
    
    # keywords with right amount of spacing
    facet = '  facet' # two spaces before
    outerloop = '    outer loop' # four spaces before, two words
    vertex = '      vertex ' # six spaces before, one after
    endloop = '    endloop' # four spaces before, one word
    endfacet = '  endfacet' # two spaces before, one word
    
    lines.extend((facet, outerloop, vertex + str(A)[1:-1], vertex + str(C)[1:-1], vertex + str(D)[1:-1], endloop, endfacet))
    lines.extend((facet, outerloop, vertex + str(A)[1:-1], vertex + str(D)[1:-1], vertex + str(B)[1:-1], endloop, endfacet))
    
    

In [57]:
def make_file(lines): # lines of the file
    
    with open('m2.stl', 'w') as f: # AUDREY EVENTUALLY CHANGE NAME TO SOMETHING NICE
        f.write('\n'.join(lines))
    

# Define Mesh - audrey you're working on this

In [29]:
# build the sampling mesh

def define_shape(resolution = 100, radius = 100.0): # AUDREY FIX THESE MAKE THEM GET USED
    # AUDREY: n should be set to resolution, protrusion added here ???  
    '''
    resolution: int
        how many facets along one edge of the original icosahedron 
    radius: float, optional
        radius of base sphere in mm (default 100mm for now)
    '''    
    # SPHERICAL COORDS OF VERTICES - easier for me to deal with later bc I can just change radius!!
    # ORDER: (r, 0, φ) defintions: 0 is right-hand sweep from x to y, φ is angle down from upwards z
    # I made these coordinates
    v0  = np.array([radius,   0, 180]) # base
    v1  = np.array([radius,   0, 120])
    v2  = np.array([radius, 324,  60])
    v3  = np.array([radius,  72, 120])
    v4  = np.array([radius,  36,  60])
    v5  = np.array([radius, 144, 120])
    v6  = np.array([radius, 108,  60])
    v7  = np.array([radius, 216, 120])
    v8  = np.array([radius, 180,  60])
    v9  = np.array([radius, 288, 120])
    v10 = np.array([radius, 252,  60])
    v11 = np.array([radius,   0,   0]) # vertex
        
    apex = v11 # top of icos
    base = v0 # bottom of icos
    
    # 10 horizontal pairs, bottom to top left to right: 
    band = [[v1, v3], [v2, v4], [v3, v5], [v4, v6], [v5, v7], [v6, v8], [v7, v9], [v8, v10], [v9, v1], [v10, v2]] 
    # hot damn I think I can populate the sphere without even using the top or botom of the icos in calculations
    
    n = resolution # sub triangle density per icos vertex (1 would correspond to just the icos) # AUDREY SET TO RESOLUTION
    
    # all the steps will be the same, so calculate it once: 
    theta_step = (v3[1] - v1[1]) / n # calculate one step in theta here (distance / #steps)
    phi_step = 180 / (3*n)
    
    big_squares = np.zeros(10, dtype = np.ndarray) # store (n x n) local population squares in here, concat later
    
    for i in range(len(band)): # there are 10 sub bands ("vectors") in the list that defines the icos middle band  
        #   A . B
        #    .   .   <= my parallelogram :)
        #     C . D
    
        A = band[i][0] # left point
        B = band[i][1] # right point  
        
        pop = np.zeros((n, n), dtype = np.ndarray) # make a temp 2D grid to be populated, local population
          
        pop[0][0] = A # set grandparent/base node to be first entry
        
        # calculate the diagonal entries:
        for ii in range(n): # (r, 0, φ)
            theta = (A[1] + theta_step * ii) % 360 # starting (left) angle (0a) with n equal spacings to the next point's 0.
            phi = (A[2]) % 180   # both A and B have the same altitude/same φ, so it doesn't matter which one I choose
            pop[ii][ii] = [1, theta, phi] # fill in the diagonal with the points equally spaced on the horizontal line
            
        for ii in range(n): 
            parent = pop[ii][ii] # the rest of the entries in the mesh will be calculated from the horizontal parent nodes
            
            # children above: parent index ii has ii children above it. 
            # node 0 is the grandparent node. it is on a vertex of the icos and is at the top left of matrix. 
            for jj in range (0, ii):
                
                # AUDREY TRY MESSING WITH THIS THETA !!! and the lower theta too if it does anything
                theta = (parent[1] - abs(jj - ii) * 0.5 * theta_step) % 360 # distance from diag corresponds to half step
                phi = (parent[2] - phi_step * (ii - jj)) % 180 # distance from diagonal corresponds to one phi step
                pop[jj][ii] = [1, theta, phi] # first index used to be ii - jj but idt that's right
                
            # children below: parent index ii has (n - 1) - ii children below it. 
            # the sum of parent's index plus number of children is (n - 1) !
            for jj in range(ii + 1, n): # here we skip the last parent because it has no lower kids, being at (n,n)
                
                theta = (parent[1] + abs(jj - ii) * 0.5 * theta_step) % 360 
                phi = (parent[2] + phi_step * (jj - ii)) % 180
                pop[jj][ii] = [1, theta, phi] 
             
        big_squares[i] = pop # fill big squares temp array with our ten local pop arrays                
       
    big_cols = np.zeros(5, dtype = np.ndarray) # store the five big columns of the grid array, made of big squares
    
    for i in range (5): # fill in the big grid array in chunks
        top = big_squares[2 * i + 1] # odd indices in big_squares correspond to top row of big grid array
        bottom = big_squares[2 * i] # even indices in big_squares correspond to bottom row of grid
        big_cols[i] = np.concatenate((top, bottom), axis = 0, dtype = np.ndarray) # build 2n tall column
        
    return apex, base, big_cols # the bulk of the shape, and then the top and bottom of the icos
    # interesting that the total number of points is 10n^2 + 2 (with n being the resolution)
            

# STL Constructor - work on this next

In [56]:
# STL constructor -- THIS IS A REWRITE !!!

def constructor(mesh = mesh, apex = apex, base = base):
    
    n = len(mesh[0][0]) # should be 4 rn # AUDREY CHANGE NAME TO RESOLUTION 
    print(n)
    
    lines = [] # initiate list to hold lines of the STL file
    lines.append('solid') # first line is "solid"
    
    for ii in range (len(mesh)): # 5 big columns
        
        for jj in range (0, 2*n - 1): # jj from 0 to 2*n - 2
            
            for kk in range (0, n - 1): # kk from 0 to n - 2
            
                # do the general logical case. safe case.
                A = mesh[ii][jj][kk] # calcualte A every time we change kk value
                B = mesh[ii][jj][kk + 1]
                C = mesh[ii][jj + 1][kk]
                D = mesh[ii][jj + 1][kk + 1]
                
                # AUDREY LOOP THE TRIANGLES EVERY TIME WE CREATE A SET OF A, B, C, D, AND RIGHT BEFORE WE CHANGE kk
                stitch_triangles(A, B, C, D, lines)
        
            kk = n - 1 # manually set kk for the weird right column cases
            A = mesh[ii][jj][kk] # new kk, new A
            
            # CASE: OPEN TOP CORNER
            if (jj < n): # last col of segment, and in the part that sticks above the next segment
                
                # SUBCASE: VERTEX CASE
                if (jj == 0): # every segment's top right most entry must connect to the icos vertex 
                    B = apex # AUDREY swap the apex and base if the file looks werid !!
                
                # SUBCASE: "GENERAL" OPEN TOP CORNER CASE
                else: 
                    B = mesh[(ii + 1) % 5][0][n - jj] 
                
                # last two points calculated the same way for all OPEN CORNER
                C = mesh[ii][jj + 1][n - 1]
                D = mesh[(ii + 1) % 5][0][n - 1 - jj]
            
            # CASE: SEGMENT SPANNING
            else: # (jj >= n)
                B = mesh[(ii + 1) % 5][jj - n][0]
                C = mesh[ii][jj + 1][n - 1]
                D = mesh[(ii + 1) % 5][jj - n + 1][0]
        
            # AUDREY LOOP TRIANGLES HERE, NOW WE'RE GONNA CALCULATE NEW INDICES
            stitch_triangles(A, B, C, D, lines)
            
            
        # CASE: BOTTOM ROW
        jj = 2*n - 1 # manually set jj for the weird bottom row cases
        
        # SUBCASE: VERTEX CASE
        kk = 0 # manually set kk for vertex case
        
        A = mesh[ii][jj][kk] # new kk, new A
        B = mesh[ii][2*n - 1][1] # AUDREY MAKE IT WORK WITH RESOLUTION = 1
        C = base # make one triangle that connects with the vertex
        D = mesh[(ii + 1) % 5][2*n - 1][0]
        
        # AUDREY STITCH TRIANGLES
        stitch_triangles(A, B, C, D, lines)
        
        # SUBCASE: "GENERAL" OPEN BOTTOM CORNER CASE
        for kk in range (1, n - 1): # will go from 1 to n - 2
            A = mesh[ii][jj][kk] # new kk, new A
            B = mesh[ii][jj][kk + 1]
            C = mesh[ii][n - kk][kk]
            D = mesh[ii][n - kk - 1][kk] 
            
            # AUDREY STITCH TRIANGLES
            stitch_triangles(A, B, C, D, lines)
        
        # SUBCASE: BOTTOM RIGHT "WEIRD FLAT TRIANGLE" CASE
        kk = n - 1 # manually set kk for the bottom right column case
            
        A = mesh[ii][jj][kk] # new kk, new A    
        B = mesh[(ii + 1) % 5][n - 1][0] # requires three points (B, D, C) to be in a line
        C = mesh[(ii + 1) % 5][n][0]
        D = mesh[(ii + 1) % 5][n + 1][0]
        
        # AUDREY STITCH TRIANGLES
        stitch_triangles(A, B, C, D, lines)
       
    lines.append('endsolid') # last line of the STL file
    
    make_file(lines) # writes the file and downloads it


# AUDREY GO MODULO EVERY INDEX JUST IN CASE OF SPILLOVERS FOR LOW RESOLUTIONS ??? maybe unnecessary let's find out
                    
                

# - - - - - - - - - - - - - - - - - - - - - - R E N D E R - - - - - - - - - - - - - - - - - - - - - -

In [50]:
# load pic
mercury0 = 'pics/mercury1.png'

In [59]:
# scale pic height values
img = scale_data()

In [60]:
# define points of a mesh of a certain radius and resolution
apex, base, big_cols = define_shape()

In [61]:
# drape the image height data onto the mesh
mesh, apex, base = data_overlay(image = img)

x: 23040 y: 11520


In [62]:
# construct and write an STL file
constructor()

100


this is a divider heheheh
-
-
-
-
-
-
-
-
-

below is reference 





In [ ]:
# AUDREY NOTES:
# PUT A RANDOM PICTURE IN
# LOOK AT THETA SCALING IN THAT ONE PLACE
# LOOK AT EXISTING PYTHON SPHERE MESH !!

In [ ]:
    ''' 
    A function that plots image data from a 2D array and optional display parameters. Returns figure and axis objects of plot.
    
    Parameters
    ----------
    image: Array
        2D array of image color values to be plotted
    figsize: int tuple, optional 
        x and y dimensions of the figure size (default (15,13))
    cmap: String, optional 
        color map for the image plotted (default = 'gray_r')
    scale: double, optional 
        image scale factor used to determine vmin and vmax if they are not provided (default = 0.5)
    colorbar: bool, optional
        include a color for the image plot (default False)
    header: Header, optional
        Header object corresponding to the image (default None)
    wcs: WCS, optional
        World Coordinate System corresponding to the image (default None)
    grid: bool, optional
        include a grid overlaid on the image (default False)
    **kwargs: dict, optional
        keyword arguments for optional plot features
        
    Returns
    -------
    plot_comps: tuple 
        Tuple of the figure and axis plot components created
    '''